In [3]:
import os
import pandas as pd
import time
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer

In [1]:
DF_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\parsedXMLs_first_predate\


In [5]:
# Import list of csvs to parse
list_csvs = []

for file in os.listdir(DF_input):
    if file[-4:] == ".csv":
        list_csvs.append(file)

n_csvs = len(list_csvs)

In [7]:
# Manage log of parsed files
path_parsed_csvs_cancer = DF_input+"csv files with Cancer and Species binarized.txt"
if not os.path.exists(path_parsed_csvs_cancer):
    open(path_parsed_csvs_cancer, 'a').close()

parsed_csvs_cancer = []
with open(path_parsed_csvs_cancer, "r") as f:
    for line in f:
        parsed_csvs_cancer.append(f)
    
for csv in tqdm(list_csvs):
    if csv not in parsed_csvs_cancer:
        start = time.time()
        print(f"Parsing csv: {csv}, {list_csvs.index(csv)+1} / {n_csvs}")

        # Import csv
        df = pd.read_csv(DF_input + csv)

        ## Binarize Cancer types
        # Use Multilabel binarizer from scikit-learn
        mlb = MultiLabelBinarizer()
        
        # Use the binarizer on the Cancer column
        df_one_hot_cancer = pd.DataFrame(
            mlb.fit_transform(df['Cancer'].astype(str)
                              .str.strip("[]")
                              .str.split(",")
                              .map(lambda x: [i.strip() for i in x])),
            columns=mlb.classes_,
            index=df.index)
        
        ## Rename the columns 
        # First column contains value 1 for articles with no found cancer
        df_one_hot_cancer.columns.values[0] = "Undetermined_Cancer"

        # Remove the '' from the names of the cancers
        for col in df_one_hot_cancer.columns[1:]:
            df_one_hot_cancer.rename(columns={col: col.strip("'")}, inplace=True)

        
        ## Binarize Species
        # Use Multilabel binarizer from scikit-learn
        mlb = MultiLabelBinarizer()
        
        # Use the binarizer on the Cancer column
        df_one_hot_species = pd.DataFrame(
            mlb.fit_transform(df['Species'].astype(str)
                              .str.strip("[]")
                              .str.split(",")
                              .map(lambda x: [i.strip() for i in x])),
            columns=mlb.classes_,
            index=df.index)
        
        ## Rename the columns 
        # First column contains value 1 for articles with no found species
        df_one_hot_species.columns.values[0] = "Undetermined_Species"

        # Remove the '' from the names of the species
        for col in df_one_hot_species.columns[1:]:
            df_one_hot_species.rename(columns={col: col.strip("'")}, inplace=True)

        # Concatenate the data with the binarized columns
        df_save = pd.concat([df, df_one_hot_cancer, df_one_hot_species], axis=1)

        # Save df, add to parsed list, clean workspace
        df_save.to_csv(DF_input+csv, index=False)
        del df, df_one_hot_cancer, df_one_hot_species, mlb, df_save
        parsed_csvs_cancer.append(csv)
        with open(path_parsed_csvs_cancer, "a") as f:
                f.write(csv+"\n")
        print(f"--Parsing time: {round(time.time()-start, 2)}")

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Parsing csv: parsedXMLs_first_pred_26400.csv, 1 / 1


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.75s/it]

--Parsing time: 1.75
